In [18]:
import httpx
import json
import os
from typing import List
import sys
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
import magic

from smart_chunker.chunker import SmartChunker
from tqdm import tqdm
import torch
import numpy as np
from smart_chunker.sentenizer import split_text_into_sentences
from transformers import AutoTokenizer, AutoModel
import os
from typing import List

In [2]:
docs_config_dir = "doc_base/RuBQ_2.0/"
test_queries_path = os.path.join(docs_config_dir, 'RuBQ_2.0_test.json')
dev_queries_path = os.path.join(docs_config_dir, 'RuBQ_2.0_dev.json')
paragraphs_path = os.path.join(docs_config_dir, 'RuBQ_2.0_paragraphs.json')

In [3]:
with open(test_queries_path, 'r', encoding='utf-8') as f:
    test_queries = json.load(f)

with open(dev_queries_path, 'r', encoding='utf-8') as f:
    dev_queries = json.load(f)

with open(paragraphs_path, 'r', encoding='utf-8') as f:
    paragraphs = json.load(f)

In [4]:
len(test_queries)

2330

In [6]:
len(test_queries)

2330

In [27]:
paragraphs

[{'uid': 0,
  'ru_wiki_pageid': 58311,
  'text': 'ЦСКА — советский и российский профессиональный хоккейный клуб из Москвы, выступающий в Континентальной хоккейной лиге. Основан в 1946 году под названием ЦДКА (Центральный дом Красной Армии). В 1951 году переименован в ЦДСА (Центральный дом Советской Армии), а в 1954 в ЦСК МО (Центральный спортивный клуб Министерства обороны), под которым выступал до 1959 года, и с тех пор носит название ЦСКА (Центральный Спортивный Клуб Армии).'},
 {'uid': 1,
  'ru_wiki_pageid': 58311,
  'text': 'В первом сезоне в составе Континентальной хоккейной лиги ЦСКА выиграл дивизион Тарасова, но в плей-офф с трудом обыграл «Ладу» (3-2 по сумме встреч) и всухую проиграл «Динамо» (0-3). В конце сезона тренерский тандем Быков-Захаркин покинул команду, аргументировав своё решение желанием сосредоточиться на работе в сборной России, однако уже через несколько недель подписали контракт с командой «Салават Юлаев», таким образом продолжив совмещать работу в сборной и в 

In [8]:
search_id = 40178
doc_texts = [par['text'] for par in paragraphs if par['ru_wiki_pageid'] == search_id]
print('\n\n'.join(doc_texts))

В сентябре 1992 года ЦСКА стартовал в Лиге чемпионов. В первом матче армейцы играли против исландского клуба «Викингур», первый матч закончился со счётом 1:0, а второй 4:2 в пользу москвичей. А в следующем матче армейцы встречались с действующим на то время обладателем Кубка чемпионов — «Барселоной». Первый матч прошёл 21 октября 1992 года в присутствии 40000 зрителей и завершился со счётом 1:1. У армейцев отличился Александр Гришин, а у испанцев — Бегиристайн. Второй матч прошёл 4 ноября 1992 года, на знаменитом домашнем стадионе «Барселоны» — «Камп Ноу», в присутствии 80000 болельщиков. Хозяева открыли счёт в матче уже на 12 минуте игры, отличился Надаль, а на 31 минуте Бегиристайн удвоил счёт в матче. За несколько минут до конца первого тайма Евгений Бушманов сумел обыграть вратаря «Барсы» Субисаррету и сократил разрыв в счёте. После перерыва, на 57 минуте, Денис Машкарин сравнял счёт. А ещё через 4 минуты Дмитрий Карсаков, после паса Ильшата Файзулина установил окончательный счёт в

### Загрузка вики-страниц для параграфов

In [ ]:
def get_text_files(src_dir: str):
    if not os.path.isdir(src_dir):
        raise Exception(f"source dir - {src_dir} doesn't exists")
    
    f = magic.Magic(mime=True)
    return [os.path.join(src_dir, f_name) for f_name in os.listdir(src_dir) if f.from_file(os.path.join(src_dir, f_name)).startswith('text/plain')]    


def parse_regular_table(table, soup_object, table_format='md'):
    # заголовки только из первой строки
    first_row = table.find('tr')
    headers = [' ' + th.get_text(strip=True) + ' ' for th in first_row.find_all('th')]
    dashes = ['-' * len(h) for h in headers]

    rows = []
    for tr in table.find_all('tr')[1:]:
        tds = tr.find_all('td')
        if not tds:
            continue
        row = []
        for td in tds:
            colspan = int(td.get('colspan', 1))
            text = ' ' + td.get_text(" ", strip=True) + ' '
            row.extend([text] * colspan)
        rows.append(row)

    # собираем Markdown
    table_data = '|' + '|'.join(headers) + '|'
    if table_format == 'md':
        dashes_line = '|' + '|'.join(dashes) + '|'
        table_data += '\n' + dashes_line
    table_data += '\n' + '\n'.join('|' + '|'.join(r) + '|' for r in rows)

    new_div = soup_object.new_tag('div')
    new_div.string = table_data
    table.replace_with(new_div)


def parse_infobox_table(table, soup_object):
    """Parse Wikipedia infobox table format."""
    rows = table.find_all('tr')
    infobox_data = []
    
    for row in rows:
        # Get all cells (both th and td)
        cells = row.find_all(['th', 'td'])
        
        if len(cells) == 2:
            # Standard key-value pair
            key = cells[0].get_text(" ", strip=True)
            value = cells[1].get_text(" ", strip=True)

            if key and value:
                infobox_data.append(f"{key} - {value}")
        elif len(cells) == 1:
            # Single cell (title, section header, etc.)
            text = cells[0].get_text(strip=True)
            if text and len(text) > 1:
                infobox_data.append(text)
    
    table_data = '\n'.join(infobox_data)
    new_div = soup_object.new_tag('div')
    new_div.string = table_data

    table.replace_with(new_div)


def extract_table_data(soup_object, table_format='md'):
    tables = soup_object.find_all('table')
          
    for table in tables:
        # check table type:
        try:
            table_classes = table.get('class', [])
        except (AttributeError, TypeError):
            table_classes = []

        if any('infobox' in table_cls.lower() for table_cls in table_classes):
            parse_infobox_table(table, soup_object)
        else:
            parse_regular_table(table, soup_object, table_format)

def drop_tables(soub_object):
    tables = soub_object.find_all('table')

    for table in tables:
        table.decompose()

def clear_html_doc(data: str, parse_tables:bool=False):
    def drop_html_artifacts(text: str) -> str:
        text = re.sub(r'\xa0', ' ', text)
        text = re.sub(r'\t', ' ', text)
        text = re.sub(r'(\s){2,}', r'\1', text)
        text = re.sub(r'\n{3,}', '\n\n', text)
        return text.strip()
    
    soup = BeautifulSoup(data, 'lxml')  # lxml parser for performance
    # drop garbage
    unwanted_selectors = [
        'script', 'style', 'noscript',  # Scripts and styles
        '.mw-jump-link',  # Skip to content links
        '#mw-navigation', '#p-search', '#p-tb', '#p-lang',  # Navigation elements
        '.navbox', '.navigation-box',  # Navigation boxes
        '.printfooter',  # Print footer
        '.catlinks',  # Category links
        '.mw-editsection',  # Edit section links
        '.reference',  # Reference links (we'll handle these differently)
        '.mw-cite-backlink',  # Citation backlinks
        '#coordinates',  # Coordinate info
        '.metadata',  # Metadata
        '.dablink',  # Disambiguation links
        '.hatnote',  # Hat notes
        '.ambox',  # Article message boxes
        '#toc',  # Table of contents
    ]
    
    # Remove unwanted elements:
    for selector in unwanted_selectors:
        for element in soup.select(selector):
            element.decompose()
    
    # parsing tables:
    if parse_tables:
        extract_table_data(soup)
    else:
        drop_tables(soup)

    content_area = soup.find('body')

    if not content_area:
        return ""
    
    # Configurable rules
    skip_section_keywords = {
        "см. также", "см также", "примечания", "ссылки", "источники", "литература"
    }
    skip_class_prefixes = ("navbox", "navigation", "references")  
    clean_text = lambda x: x.get_text(strip=True)

    for el in content_area.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6',
                                     'p', 'div', 'li', 'dd', 'dt']):
        tag = el.name
        text = clean_text(el) 

        if not text:
            continue

        # headers:
        if tag.startswith("h"):
            text_clean = text.lower()

            text_clean = el.get_text(strip=True).lower()
            if any(text_clean.startswith(skip) for skip in skip_section_keywords):
                # удаляем этот заголовок и всё, что после него
                for sibling in list(el.find_all_next()):
                    sibling.decompose()
                el.decompose()
                break
        elif tag == "div":
            classes = el.get("class", [])
            el_ids = el.get("id", "").split()

            if any(cl.startswith(prefix) for cl in classes + el_ids for prefix in skip_class_prefixes):
                el.decompose()
          
    return drop_html_artifacts(content_area.get_text())    


def load_source_pages(paragraphs: List[dict], 
                      dst_dir: str='doc_base/wiki_docs/', 
                      continue_downloading: bool = True, 
                      single_file_timeout: int = 30,
                      follow_redirects: bool = True):
    get_file_id  = lambda x: int(os.path.basename(x).split('.')[0])

    def get_downloaded_files_ids():
        return set(get_file_id(file) for file in get_text_files(dst_dir))
        

    def scrap_page(wiki_url: str, wiki_page_id: int):
        resp = httpx.get(wiki_url, timeout=single_file_timeout, follow_redirects=follow_redirects)
        resp.raise_for_status()
        data = resp.text
        data = clear_html_doc(data)

        with open(os.path.join(dst_dir, f'{wiki_page_id}.txt'), 'w', encoding='utf-8') as f:
            f.write(data)

    if not os.path.isdir(dst_dir):
        os.mkdir(dst_dir)
        
    meta_data = dict()
    error_pages = []
    pages_set = set([item['ru_wiki_pageid'] for item in paragraphs])
    pbar = tqdm(total=len(pages_set))
    existing_ids = get_downloaded_files_ids() if continue_downloading else set()
    pbar.update(len(existing_ids))

    try:
        for item in paragraphs:
            wiki_page_id = item["ru_wiki_pageid"]

            if wiki_page_id not in existing_ids:
                # page is not loaded yet
                wiki_url = f'https://ru.wikipedia.org/w/index.php?curid={wiki_page_id}'

                try:
                    scrap_page(wiki_url, wiki_page_id)
                    
                except httpx.RequestError as req_exc:
                    msg = f'request error for url={wiki_url}: {str(req_exc)}'
                    print(msg, file=sys.stderr)
                    error_pages.append({'url': wiki_url, 'exc_type': 'request error', 'msg': msg})
                except httpx.HTTPStatusError as st_exc:
                    msg = f'invalid status for url={wiki_url}: {str(st_exc)}'
                    print(msg, file=sys.stderr)
                    error_pages.append({'url': wiki_url, 'exc_type': 'status error', 'msg': msg})
                pbar.update(1)
                existing_ids.add(wiki_page_id)
            meta_data.setdefault(wiki_page_id, []).append(item['uid'])

    except KeyboardInterrupt as e:
        print(f'Downloading has been interrupted', file=sys.stderr)
    finally:
        # write exceptions info:
        with open(os.path.join(dst_dir, 'load_errors.json'), 'w', encoding='utf-8') as f:
            f.write(json.dumps(error_pages))

        # write metadata info:
        meta_data_json = []
        for key, val in meta_data.items():
            wiki_url = f'https://ru.wikipedia.org/w/index.php?curid={key}'
            item = {'url': wiki_url, 'page_id': key, 'paragraphs_ids': val}
            meta_data_json.append(item)

        with open(os.path.join(dst_dir, 'meta_data.json'), 'w', encoding='utf-8') as f:
            f.write(json.dumps(meta_data_json))

In [43]:
load_source_pages(paragraphs)

 86%|████████▌ | 7847/9105 [00:23<00:03, 338.72it/s]invalid status for url=https://ru.wikipedia.org/w/index.php?curid=3682344: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=3682344'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
invalid status for url=https://ru.wikipedia.org/w/index.php?curid=5705399: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=5705399'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
invalid status for url=https://ru.wikipedia.org/w/index.php?curid=7538508: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=7538508'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
invalid status for url=https://ru.wikipedia.org/w/index.php?curid=4323198: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=4323198'
For more information 

In [12]:
link1='https://ru.wikipedia.org/w/index.php?curid=14614'
link2='https://ru.wikipedia.org/w/index.php?curid=1787424'
link3='https://ru.wikipedia.org/w/index.php?curid=4336008'

In [13]:
def scrap_page(wiki_url: str, wiki_page_id:int, dst_dir:str='doc_base/wiki_docs/'):
    resp = httpx.get(wiki_url, timeout=30, follow_redirects=True)
    resp.raise_for_status()
    data = resp.text
    data = clear_html_doc(data)

    with open(os.path.join(dst_dir, f'{wiki_page_id}.txt'), 'w', encoding='utf-8') as f:
        f.write(data)

scrap_page(link1, 14614)
scrap_page(link2, 1787424)
scrap_page(link3, 4336008)


In [15]:
def get_file_data(doc_id:int, dst_dir:str='doc_base/wiki_docs/'):
    with open(f'{dst_dir}/{doc_id}.txt', encoding='utf-8') as f:
        return f.read()

In [18]:
get_file_data(4336008)

'Премия «Хьюго» за лучший рассказ\nМатериал из Википедии — свободной энциклопедии\nТекущая версия страницы пока не проверялась опытными участниками и может значительно отличаться от версии, проверенной 19 сентября 2019 года; проверки требуют 84 правки.\nПремия «Хьюго» за лучший рассказ (англ. Hugo Award for Best Short Story) ежегодно вручается с 1955 года на Всемирном конвенте любителей фантастики «Worldcon», кроме 1957 года, за лучшие произведения, написанные в жанре научной фантастики или фэнтези и опубликованные или переведённые на английский язык в предыдущем календарном году. Художественное произведение определяется организаторами премии как рассказ, если его текст содержит менее 7500 слов. Лауреатам вручается статуэтка, изображающая взлетающую ракету.\nНоминантов и победителей выбирают зарегистрированные участники конвента «Worldcon», вечерняя презентация которого является его центральным событием. Процесс отбора проводится методом преференциального голосования с пятью номинантам

### Чанкинг

In [ ]:
def split_docs_on_chunks(src_dir: str, dst_dir: str, delimiter:str='\n'*4):
    if not os.path.isdir(dst_dir):
        os.mkdir(dst_dir)
    text_files = get_text_files(src_dir)

    chunker = SmartChunker(
        language='ru',
        reranker_name='BAAI/bge-reranker-v2-m3',
        newline_as_separator=False,
        device='cuda:0'
    )

    pbar = tqdm(len(text_files), desc='splitting docs into chunks...')
    for f_path in text_files:
        with open(f_path, encoding='utf-8') as f:
            data = f.read()
        chunks = chunker.split_into_chunks(data)
        with open(os.path.join(dst_dir, os.path.basename(f_path)), encoding='utf-8') as f:
            f.write(delimiter.join(chunks))
        pbar.update(1)

    del chunker


def split_docs_on_chunks_bi_encoder(src_dir: str, dst_dir: str, delimiter:str='\n'*4):
    if not os.path.isdir(dst_dir):
        os.mkdir(dst_dir)
    text_files = get_text_files(src_dir)

    def calculate_document_embedding(document: str, embedder) -> np.ndarray:
        tokenizer = embedder[0]
        model = embedder[1]
        tokenized_inputs = tokenizer([document], max_length=512, padding=True, truncation=True, return_tensors='pt').to(model.device)
        with torch.no_grad():
            outputs = model(**tokenized_inputs)

        embeddings = torch.nn.functional.normalize(outputs[0][:, 0], p=2, dim=1).cpu().numpy()
        return embeddings

    def join_chunks_by_semantics(chunks: List[str], emb_tokenizer, similarities_between_neighbors: np.ndarray, max_tokens:int=256) -> List[str]:
        if len(chunks) < 2:
            return chunks
        n_tokens = len(emb_tokenizer.tokenize('\n'.join(chunks)))
        if n_tokens <= max_tokens:
            return ['\n'.join(chunks)]
        min_similarity_idx = 0
        for idx in range(1, len(chunks) - 1):
            if similarities_between_neighbors[idx] < similarities_between_neighbors[min_similarity_idx]:
                min_similarity_idx = idx
        if min_similarity_idx == 0:
            res = [chunks[0]]
        else:
            res = join_chunks_by_semantics(chunks[:(min_similarity_idx + 1)], emb_tokenizer, similarities_between_neighbors[:min_similarity_idx])
        if min_similarity_idx == (len(chunks) - 2):
            res += [chunks[-1]]
        else:
            res += join_chunks_by_semantics(chunks[(min_similarity_idx + 1):], emb_tokenizer, similarities_between_neighbors[(min_similarity_idx + 1):])
        return res
    
    def join_chunks(file_path:str, model, tokenizer):
        with open(file_path, 'r', encoding='utf-8') as f:
            file_text = f.read()
        chunks = split_text_into_sentences(file_text)
        embeddings = np.vstack([calculate_document_embedding(it, (tokenizer, model)) for it in chunks])
        similarities = np.array([(embeddings[idx:(idx + 1)] @ embeddings[(idx + 1):(idx + 2)].T).tolist()[0] for idx in
                                range(len(chunks) - 1)]).reshape((len(chunks) - 1,))
        return join_chunks_by_semantics(chunks, tokenizer, similarities)
    
    tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3")
    model = AutoModel.from_pretrained("deepvk/USER-bge-m3")
    model.eval()

    pbar = tqdm(len(text_files), desc='splitting docs into chunks...')
    for f_path in text_files:
        chunks = join_chunks(f_path, model, tokenizer)
        with open(os.path.join(dst_dir, os.path.basename(f_path)), 'w', encoding='utf-8') as f:
            f.write(delimiter.join(chunks))
        pbar.update(1)

    del model
    del tokenizer

In [12]:
split_docs_on_chunks_bi_encoder('/home/cossmo/RAG/doc_base/wiki_docs', '/home/cossmo/RAG/doc_base/wiki_docs_chunked')

Token indices sequence length is longer than the specified maximum sequence length for this model (14437 > 8192). Running this sequence through the model will result in indexing errors
splitting docs into chunks...: 3it [05:05, 101.88s/it]


In [37]:
dst_dir = 'doc_base/wiki_docs'
text_files = get_text_files(dst_dir)
print(f'text files count={len(text_files)}')
print(f'files in dir={len(os.listdir(dst_dir))}')
print(f'')

text files count=7847
files in dir=7847



In [36]:
text_set = set(text_files)
files_set = set([os.path.join(dst_dir, file) for file in os.listdir(dst_dir)])
files_set - text_set

set()

In [35]:
for file in files_set - text_set:
    os.remove(file)
    print(f'{file} removed')

doc_base/wiki_docs/74257.txt removed
doc_base/wiki_docs/74302.txt removed
doc_base/wiki_docs/3675114.txt removed
doc_base/wiki_docs/193.txt removed
doc_base/wiki_docs/8509.txt removed
doc_base/wiki_docs/74417.txt removed
doc_base/wiki_docs/74446.txt removed


In [44]:
import os

print([item for item in os.listdir('doc_base/wiki_docs') if item.split('.')[-1]=='json'])

['load_errors.json', 'meta_data.json']


In [1]:
import json


with open(f'doc_base/wiki_docs/load_errors.json', encoding='utf-8') as f:
    data = f.read()
    errors = json.loads(data)
errors

[{'url': 'https://ru.wikipedia.org/w/index.php?curid=3682344',
  'exc_type': 'status error',
  'msg': "invalid status for url=https://ru.wikipedia.org/w/index.php?curid=3682344: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=3682344'\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404"},
 {'url': 'https://ru.wikipedia.org/w/index.php?curid=5705399',
  'exc_type': 'status error',
  'msg': "invalid status for url=https://ru.wikipedia.org/w/index.php?curid=5705399: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=5705399'\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404"},
 {'url': 'https://ru.wikipedia.org/w/index.php?curid=7538508',
  'exc_type': 'status error',
  'msg': "invalid status for url=https://ru.wikipedia.org/w/index.php?curid=7538508: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=7538508'\nFor more 

### Drop invalid paragraphs

In [ ]:
for item in errors:

### Open router price

In [4]:
metrics_count = 4 # Faithfulness, Response Relevancy, Context Precision, Context Recall
context = 4e3
input_price = 0.3
output_price = 0.3
benchmark_size = 1000
input_fraction = 0.3
rag_config_count = 10

total_price = rag_config_count * metrics_count * context * benchmark_size * (input_fraction * input_price + (1 - input_fraction) * output_price) / 1e6
print(f'total price in USD={total_price:.2f}')

total price in USD=48.00


### Yandex Cloud price

In [ ]:
# YANDEX CLOUD PRICE
metrics_count = 4 # Faithfulness, Response Relevancy, Context Precision, Context Recall
context = 8e3
benchmark_size = 1000
input_fraction = 0.1
token_price = 0.4 # Qwen3-30B-A3B
total_price = metrics_count * context * benchmark_size * token_price / 1e3
print(f'total price in RUB={total_price:.2f}')

total price in RUB=12800.00


### Check Open Router API

In [1]:
token_path = 'token'

with open(token_path, 'r', encoding='utf-8') as f:
    token = f.read()

In [6]:
from openai import OpenAI


client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=token,
)

completion = client.chat.completions.create(
    model="qwen/qwen3-next-80b-a3b-instruct",
    messages=[
        {"role": "user", "content": "Привет! А что ты за модель? Сколько ты весишь?"}
    ],
    max_completion_tokens = 1024,
    extra_body={
        "provider": {
            #"order": ["Hyperbolic", "GMICloud", "DeepInfra"],
            "only":["Hyperbolic"],
            "quantizations": ["bf16"],
            "allow_fallbacks": False,
            "require_parameters": True
        }
    }
)

In [7]:
print(completion.choices[0].message.content)

Привет! 😊  
Я — Qwen, большая языковая модель, разработанная Alibaba Cloud. Я не имею физической формы, поэтому у меня нет веса в привычном смысле — как у человека или предмета. Но если говорить о «весе» в цифровом смысле, то мой размер (количество параметров) может достигать сотен миллиардов, что делает меня одной из самых мощных моделей в своём классе! 🚀

Я могу помогать с ответами на вопросы, созданием текстов, логическими рассуждениями, программированием и даже шутками — что угодно! Чем могу помочь прямо сейчас? 💬


### Test Ragas RAG evaluation

In [ ]:
for answer in test_queries[0]['answers']:
    print(answer['label'])
    print(answer[''])

землетрясение
метеорит
оползень
Проект Seal
подводный оползень
извержение вулкана


In [17]:
def grep(search_dir, pattern: str = r".+\.json$"):
    files = os.listdir(search_dir)
    return [
        os.path.join(search_dir, file)
        for file in files
        if re.fullmatch(pattern, file)
    ]

# Usage
print(grep("."))

['./token.json']


In [48]:
def create_test_bench(bench_size, queries: list[dict], random_state = 42):
    np.random.seed(seed=random_state)
    idxs = np.arange(0, len(queries))
    np.random.shuffle(idxs) # shuffle inplace

    bench_queries = np.array(queries)[idxs][:bench_size]
    result = []

    for i, query in enumerate(bench_queries):
        item = {'uid' : query['uid'],
                'text': query['question_text'],
                'answers': [item['label'] for item in query['answers']],
                'answer_text': query['answer_text'],
                'answer_paragraphs': query['paragraphs_uids']['with_answer'],
                'relative_paragraphs': query['paragraphs_uids']['all_related']}
        result.append(item)

    return result


In [51]:
bench = create_test_bench(100, dev_queries)
bench[:1]

[{'uid': 8170,
  'text': 'Как зовут отца Витаса?',
  'answers': [],
  'answer_text': 'Владас Аркадьевич Грачёв',
  'answer_paragraphs': [],
  'relative_paragraphs': [34403,
   34404,
   34405,
   34406,
   34407,
   35014,
   35015,
   35016,
   35017,
   35018]}]

#### check answer length distribution

In [55]:
import plotly.graph_objects as go


def plot_answer_len_dist(queries):
    def get_distribution():
        return [len(query['answer_text']) for query in queries]

    tokens_count = get_distribution()

    fig = go.Figure(
        data=[go.Histogram(x=tokens_count, nbinsx=50, marker=dict(line=dict(width=1, color="black")))]
    )

    # threshold:
    fig.update_layout(
        title="Answer len distribution",
        xaxis_title="Number of chars",
        yaxis_title="Frequency",
        bargap=0.1,
    )    

    fig.show()

In [56]:
plot_answer_len_dist(dev_queries)

In [57]:
plot_answer_len_dist(test_queries)

In [62]:
max_answer = max(dev_queries, key = lambda x: len(x['answer_text']))
print(f'question: {max_answer['question_text']}')
print(f'answer: {max_answer['answer_text']}')

question: Кто участвовал в 1 мировой войне?
answer: США, Канада, Османская империя, Австралия, Австро-Венгрия, Бразилия, Бельгия, Британская империя, Таиланд, Германская империя, Финляндия, Российская империя, Горская республика, Герцогство Курляндское и Земгальское, Российская республика, Третья Французская республика, Украинская народная республика, Грузинская демократическая республика, Королевство Литва, Кубанская народная республика, Японская империя, Китайская Республика, Королевство Италия, Южно-Африканский союз, Королевство Венгрия, Королевство Румыния, Королевство Сербия, Белорусская народная республика, Третье Болгарское царство, Азербайджанская Демократическая Республика, Королевство Черногория, Джебель-Шаммар, Хиджаз, Государство дервишей, Украинская держава, Первая Португальская республика, Ньюфаундленд, Балтийское герцогство, Крымское краевое правительство, Всевеликое войско Донское, Дарфурский султанат, Доминион Новая Зеландия, Королевство Греция, Королевство Польское


Как мы видим, в подавляющем большинстве ответы односложные

In [ ]:
import pandas as pd

samples = [
    {"query": "What is Ragas 0.3?", "grading_notes": "- Ragas 0.3 is a library for evaluating LLM applications."},
    {"query": "How to install Ragas?", "grading_notes": "- install from source  - install from pip using ragas[examples]"},
    {"query": "What are the main features of Ragas?", "grading_notes": "organised around - experiments - datasets - metrics."}
]
pd.DataFrame(samples).to_csv("datasets/test_dataset.csv", index=False)

In [ ]:
from ragas.metrics import DiscreteMetric
from ragas import experiment
import asyncio
from ragas.metrics import FactualCorrectness


@experiment()
async def evaluated_experiment(item, rag_client):
    response = await rag_client.query(row["input"])

    # Calculate metrics inline
    factual_score = FactualCorrectness().score(
        response=response,
        reference=row["expected_output"]
    )

    return {
        **row,
        "response": response,
        "factual_correctness": factual_score.value,
        "factual_reason": factual_score.reason,
        "experiment_name": "evaluated_v1"
    }

NameError: name 'experiment' is not defined